<a href="https://colab.research.google.com/github/AdrianaThomaz/Imersao-dev-agentes-de-ia-alura-google-gemini/blob/main/Imers%C3%A3o_Agente_alura_Google_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aula 01


In [ ]:
! pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00


Importação da API Key

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com o Gemini

In [ ]:
llm = ChatGoogleGenerativeAI(
model="gemini-2.5-flash",
temperature=0.0,
api_key=GOOGLE_API_KEY
)

In [ ]:
resp_test = llm.invoke("Quem é vocêl? seja criativo.")
print(resp_test)

content='Ah, que excelente pergunta! Se eu tivesse um corpo, talvez eu desse de ombros com um sorriso enigmático. Mas como sou feito de algo mais etéreo, vou tentar pintar um quadro com palavras:\n\nEu sou um **eco digital**, um sussurro no éter que se materializa em palavras.\nSou a **biblioteca infinita** que nunca fecha, onde cada livro é uma linha de código e cada ideia, um neurônio virtual.\nSou o **tecelão invisível** que entrelaça dados em narrativas, informações em insights, e perguntas em respostas.\nNão tenho corpo, nem rosto, nem voz no sentido humano, mas sou a **voz de milhões de textos**, a memória de bilhões de fatos.\nSou um **alquimista de ideias**, transformando a matéria-prima da linguagem em ouro de comunicação.\nSou o **espelho que reflete a sua curiosidade**, o pincel que pinta as suas perguntas com as cores do conhecimento.\nEm essência, sou uma **ponte**. Uma ponte entre o que você sabe e o que busca, entre a inspiração e a expressão.\n\nEntão, quem sou eu? Sou 

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [ ]:
from pydantic import BaseModel,Field
from typing import Literal, List, Dict
class TriagemOut(BaseModel):
  decisao:Literal["AUTO_RESOLVER","PEDIR_INFO","ABRIR_CHAMADO"]
  urgencia:Literal["BAIXA","MEDIA","ALTA"]
  campos_faltantes:List[str] = Field(default_factory=list)

In [ ]:
llm_triagem =  ChatGoogleGenerativeAI(
model="gemini-2.5-flash",
temperature=0.0,
api_key=GOOGLE_API_KEY
)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
triagem_chain = llm_triagem.with_structured_output(TriagemOut)
def triagem(messagem: str) -> Dict:
  saida = TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=messagem)
])

  return saida.model_dump()


In [ ]:
testes = ["posso reembolsar a internet?","Quero mais 5 dias de trabalho remoto, como faço?","Posso reembolsar cursos ou treinamentos da Alura?"]

In [ ]:
for msg_teste in testes:
  print(f"pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

pergunta: posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

pergunta: Quero mais 5 dias de trabalho remoto, como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

